## Using PSNR for the loss function for Hand-written Digit Regression model
This notebook uses Fireball to creare a regression model. The input to the model is a 28x28 monochrome image from MNIST dataset. The output is the predicted value of the model as a single number. Note that the model does not classify the image. It predicts a floating point value.

In this notebook we show how to pass a user-defined loss function to the Fireball model. We use PSNR as the loss function; so, the model learns to **maximize** PSNR unlike the standard case where MSE is minimized.

Please note that Fireball even copies the loss function in the saved model. So, when the model is loaded later, the loss function does not need to be defined again. The loaded model already has a copy of user defined loss function even though it was not originally part of Fireball.

This notebook also shows how to subclass a Fireball dataset class. We want the labels in the MNIST dataset to be a floating point value. We also want the main evaluation metric of the dataset to be PSNR instead of MSE.

## Subclassing the MNIST dataset

In [1]:
import numpy as np
import os

from fireball import Model
from fireball.datasets.mnist import MnistDSet

class RegMnistDSet(MnistDSet):
    # ******************************************************************************************************************
    @classmethod
    def postMakeDatasets(cls):
        # This is called at the end of a call to makeDatasets
        cls.numClasses = 0            # Make this a regression dataset
        cls.psnrMax = 9.0             # Set the max value of the output for PSNR calculations
        cls.evalMetricName = 'PSNR'   # Set the main evaluation metric to PSNR

    # ******************************************************************************************************************
    def getBatch(self, batchIndexes):
        return self.samples[batchIndexes], np.float32(self.labels[batchIndexes]) # Return labels as float32

trainDs, testDs = RegMnistDSet.makeDatasets('train,test', batchSize=128)
RegMnistDSet.printDsInfo(trainDs, testDs)



RegMnistDSet Dataset Info:
    Dataset Location ............................... /Users/shahab/data/mnist/
    Number of Training Samples ..................... 60000
    Number of Test Samples ......................... 10000
    Sample Shape ................................... (28, 28, 1)


## Defining the loss function and creating the Fireball model

Fireball can recieve a loss function when a model is instantiated. This function is then used by fireball to calculate the loss in the training mode. This function takes the following arguments:
- layers: A "Layers" object as defined in the "Layers.py" file. This keeps a list of all the layers in the model that may be used for the calculation of the loss.
- predictions: The output(s) of the network just before the output layer. This is a tuple containing all outputs of the network.
- groundTruths: The batch of labels used for the training step. This is a tuple containing all label objects. The tuple usually contains the placeholders created in the Layer's "makePlaceholders" function.


In [2]:
import tensorflow as tf
def psnrLoss(layers, predictions, groundTruths):
    rmse = tf.sqrt( tf.reduce_mean( tf.square(predictions[0] - groundTruths[0]) ) )
    rmseClipped = tf.clip_by_value(rmse, 0.000001, 10000000.0, name='CLIP')
    psnr = 20.*tf.log(9.0/rmseClipped)/np.log(10)
    return -psnr  # We want to maximize PSNR which means minimizing -PSNR


# Here we define a "Regression LeNet-5" network which has 2 convolutional layers followed by 3 fully connected
layersInfo = ('IMG_S28_D1,' +                 # The input layer takes a 28x28 image of depth 1 (monochrome)
              'CONV_K5_O6_Ps:ReLU:MP_K2,' +   # Conv, Kernel size 5, 6 out channels, "same" padding, ReLU, Max pool
              'CONV_K5_O16_Pv:ReLU:MP_K2,' +  # Conv, Kernel size 5, 16 out channels, "valid" padding, ReLU, Max pool
              'FC_O120:ReLU,FC_O84:ReLU,FC_O1:ReLU,' +   # 3 fully connected layers
              'REG')                          # Unlike original LeNet-5, the output is just a float32 number

model = Model(name='RegMnistTest',
              layersInfo=layersInfo,
              trainDs=trainDs, testDs=testDs,
              numEpochs=10,
              regFactor=0.0001,
              learningRate=(0.001,0.0001),
              optimizer="Adam",
              lossFunction=psnrLoss,
              gpus="0")

model.printLayersInfo()
model.printNetConfig()
model.initSession()
model.train()
model.save("Models/TestCustomLoss.fbm")


Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 28x28x1      28 28 1       None                      0          
L1_CONV          28 28 1       KSP: 5 1 s               14 14 6       ReLU     MP(KSP):2 2 v    156        
L2_CONV          14 14 6       KSP: 5 1 v               5 5 16        ReLU     MP(KSP):2 2 v    2,416      
L3_FC            5 5 16                                 120           ReLU                      48,120     
L4_FC            120                                    84            ReLU                      10,164     
L5_FC            84                                     1             ReLU                      85         
OUT_REG          1                                      1             None                      0          
---------------------------

## Running inference on the trained model
Here we run inference on random samples from the test dataset. Run this several times and see the difference between the actual digit and the predicted value.

In [3]:
i = np.random.randint(testDs.numSamples)
print( "Actual label of the sample no %d in the dataset: %d"%(i, testDs.labels[i]))
print( "Predicted Value: %f"%(model.inferOne(testDs.samples[i])))

Actual label of the sample no 2468 in the dataset: 6
Predicted Value: 6.177000


## Evaluating the model
We can now call the ```evaluate``` function of the Fireball model to evaluate it as a regression problem. The standard metric for the evaluation of a regression problem is mean squared error (MSE).



In [4]:
Results = model.evaluate()

  Processed 10000 Sample. (Time: 1.77 Sec.)                              

MSE:  0.227759
RMSE: 0.477241
MAE:  0.203611
PSNR: 25.510095


## Persistance of user-defined loss function
Fireball includes a copy of customized loss function in the ```fbm``` file. As long as the loss function does not use any global info, it will continue to work after loading the file.

To test this, reset the kernel to make sure the above loss function definition is gone. Then run the only first cell above to create the dataset. DO NOT run the second cell.

In [5]:
# Now we create a new model using the saved model file. This model already "contains" our
# user-defined loss function eventhough we did not pass any loss function to it.
# Of course we could override the original loss function by passing a new one to the 
# "makeFromFile" function.
# Here we want to train it for 4 more epochs with smaller learning rate.
from fireball import Model
model = Model.makeFromFile("Models/TestCustomLoss.fbm",
                           trainDs=trainDs, testDs=testDs,
                           optimizer="Adam",
                           numEpochs=4,
                           learningRate=0.0001,
                           gpus="0")
model.initSession()
model.evaluateDSet(testDs)

# Train the model for 4 more epochs
model.train()

# Evaluate again
results = model.evaluateDSet(testDs)


Reading from "Models/TestCustomLoss.fbm" ... Done.
Creating the fireball model "RegMnistTest" ... Done.
  Processed 10000 Sample. (Time: 0.91 Sec.)                              

MSE:  0.227759
RMSE: 0.477241
MAE:  0.203611
PSNR: 25.510095
+--------+---------+---------------+-----------+-------------------+
| Epoch  | Batch   | Learning Rate | Loss      | Valid/Test PSNR   |
+--------+---------+---------------+-----------+-------------------+
| 1      | 469     | 0.0001        | -27.67975 | N/A      25.545   |
| 2      | 938     | 0.0001        | -27.83217 | N/A      25.402   |
| 3      | 1407    | 0.0001        | -27.89658 | N/A      25.633   |
| 4      | 1876    | 0.0001        | -27.94295 | N/A      25.576   |
+--------+---------+---------------+-----------+-------------------+
Total Training Time: 96.28 Seconds
  Processed 10000 Sample. (Time: 1.41 Sec.)                              

MSE:  0.224338
RMSE: 0.473643
MAE:  0.194202
PSNR: 25.575821


## Where do I go from here?
[Handwritten Digit recognition (LeNet-5/MNIST)](LeNet5/LeNet5-MNIST.ipynb)

[Reducing number of parameters of LeNet-5 Model](LeNet5-MNIST-Reduce.ipynb)

[Quantizing the LeNet-5 Model](LeNet5-MNIST-Quantize.ipynb)

[Exporting LeNet-5 Model to ONNX](LeNet5-MNIST-ONNX.ipynb)

[Exporting LeNet-5 Model to CoreML](LeNet5-MNIST-CoreML.ipynb)

[Exporting LeNet-5 Model to TensorFlow](LeNet5-MNIST-TF.ipynb)

---

[Fireball Playgrounds](../Contents.ipynb)
